# Teacher Model Training

Code authored by: Shaw Talebi

[Video](https://youtu.be/4QHg8Ix8WWQ) <br>
[Blog](https://medium.com/towards-data-science/fine-tuning-bert-for-text-classification-a01f89b179fc) <br>
Based on example [here](https://huggingface.co/docs/transformers/en/tasks/sequence_classification)

### imports

In [ ]:
from datasets import load_dataset, DatasetDict
from datasets import Dataset
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


### load data

In [3]:
import os

# ---- Data paths (edit these for your machine) ----
EXCEL_PATH = "/Users/rishirajsaikia/Downloads/ASIN Master -Sample 100.xlsx"  # optional
CSV_PATH = "/Users/rishirajsaikia/Downloads/ASIN-Sample 100.csv"

# If you have the Excel file, we create/refresh the CSV from it.
# If you only have the CSV, set EXCEL_PATH = None (or leave it missing on disk).
if EXCEL_PATH and os.path.exists(EXCEL_PATH):
    df = pd.read_excel(EXCEL_PATH)
    df.columns = df.columns.str.strip()
    df = df[["Description", "Sub  Category"]].dropna()
    df.to_csv(CSV_PATH, index=False)

# Load as a Hugging Face dataset
dataset_dict = load_dataset("csv", data_files=CSV_PATH)
# dataset_dict=dataset_dict['train'].train_test_split(test_size=0.2)
dataset_dict


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Description', 'Sub  Category'],
        num_rows: 98
    })
})

In [38]:
# Optional: quick peek at the data we loaded
df_preview = pd.read_csv(CSV_PATH)
print(df_preview.columns.to_list())
df_preview.head(4)


['Description', 'Sub  Category']


,Description,Sub Category
0,Sri Sri d shuddhta ka naam Sri Sri Tattva Prem...,DAIRY_BASED_BUTTER
1,American Tourister Poler Polyester 65 cms Blac...,DUFFEL_BAG
2,LOREM Green Removable Card Holder Bi-Fold Faux...,WALLET
3,Echt Die Cast Aluminium Non Stick Combo Set of...,COOKWARE_SET


In [4]:
# Create train/validation/test splits (80/10/10)
# NOTE: load_dataset('csv', ...) returns a DatasetDict with a single 'train' split by default.
# We split that into train/validation/test for training + evaluation.
splits = dataset_dict["train"].train_test_split(test_size=0.1, seed=42)

# Split the 20% "test" chunk into validation (10%) and test (10%)
val_test = splits["test"].train_test_split(test_size=0.5, seed=42)

dataset_dict = DatasetDict(
    {
        "train": splits["train"],
        "validation": val_test["train"],
        "test": val_test["test"],
    }
)

dataset_dict


NameError: name 'DatasetDict' is not defined

### Train Teacher Model

In [12]:
# Get unique categories and create label mappings
label_col = "Sub  Category"
text_col = "Description"

all_categories = set()
for split_name in dataset_dict.keys():
    all_categories.update(
        [str(x).strip() for x in dataset_dict[split_name].unique(label_col) if x is not None]
    )

categories = sorted(all_categories)
id2label = {i: cat for i, cat in enumerate(categories)}
label2id = {cat: i for i, cat in enumerate(categories)}

model_path = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=len(categories),
    id2label=id2label,
    label2id=label2id,
)


Flattening the indices:   0%|          | 0/78 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/10 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/10 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Freeze base model

In [13]:
# print layers
for name, param in model.named_parameters():
   print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight True
bert.embeddings.position_embeddings.weight True
bert.embeddings.token_type_embeddings.weight True
bert.embeddings.LayerNorm.weight True
bert.embeddings.LayerNorm.bias True
bert.encoder.layer.0.attention.self.query.weight True
bert.encoder.layer.0.attention.self.query.bias True
bert.encoder.layer.0.attention.self.key.weight True
bert.encoder.layer.0.attention.self.key.bias True
bert.encoder.layer.0.attention.self.value.weight True
bert.encoder.layer.0.attention.self.value.bias True
bert.encoder.layer.0.attention.output.dense.weight True
bert.encoder.layer.0.attention.output.dense.bias True
bert.encoder.layer.0.attention.output.LayerNorm.weight True
bert.encoder.layer.0.attention.output.LayerNorm.bias True
bert.encoder.layer.0.intermediate.dense.weight True
bert.encoder.layer.0.intermediate.dense.bias True
bert.encoder.layer.0.output.dense.weight True
bert.encoder.layer.0.output.dense.bias True
bert.encoder.layer.0.output.LayerNorm.weight True


In [14]:
# freeze base model parameters
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [15]:
# print layers
for name, param in model.named_parameters():
   print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight False
bert.embeddings.position_embeddings.weight False
bert.embeddings.token_type_embeddings.weight False
bert.embeddings.LayerNorm.weight False
bert.embeddings.LayerNorm.bias False
bert.encoder.layer.0.attention.self.query.weight False
bert.encoder.layer.0.attention.self.query.bias False
bert.encoder.layer.0.attention.self.key.weight False
bert.encoder.layer.0.attention.self.key.bias False
bert.encoder.layer.0.attention.self.value.weight False
bert.encoder.layer.0.attention.self.value.bias False
bert.encoder.layer.0.attention.output.dense.weight False
bert.encoder.layer.0.attention.output.dense.bias False
bert.encoder.layer.0.attention.output.LayerNorm.weight False
bert.encoder.layer.0.attention.output.LayerNorm.bias False
bert.encoder.layer.0.intermediate.dense.weight False
bert.encoder.layer.0.intermediate.dense.bias False
bert.encoder.layer.0.output.dense.weight False
bert.encoder.layer.0.output.dense.bias False
bert.encoder.layer.0.output.Lay

#### Preprocess text

In [16]:
def preprocess_function(examples):
    # Tokenize text
    texts = [t if t is not None else "" for t in examples[text_col]]
    tokenized = tokenizer(texts, truncation=True)

    # ✅ IMPORTANT: Provide integer labels so the model can compute loss during training.
    # The Trainer passes `labels=...` into the model; without this, the model returns only logits (no loss).
    tokenized["labels"] = [label2id[str(cat).strip()] for cat in examples[label_col]]

    return tokenized


In [17]:
# tokenize all datasetse
tokenized_data = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/78 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [18]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#### Evaluation

In [19]:
# Metrics (works for binary OR multiclass)
def _softmax(logits: np.ndarray) -> np.ndarray:
    logits = logits - np.max(logits, axis=-1, keepdims=True)  # stability
    exp = np.exp(logits)
    return exp / np.sum(exp, axis=-1, keepdims=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    preds = np.argmax(logits, axis=-1)
    probs = _softmax(logits)

    metrics = {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
    }

    # ROC AUC:
    # - Binary: use prob of class 1
    # - Multiclass: one-vs-rest macro average
    try:
        if probs.shape[1] == 2:
            metrics["auc"] = roc_auc_score(labels, probs[:, 1])
        else:
            metrics["auc_ovr_macro"] = roc_auc_score(labels, probs, multi_class="ovr", average="macro")
    except ValueError:
        # Can happen if a split contains only 1 class, etc.
        pass

    # Round for cleaner logs
    return {k: float(np.round(v, 4)) for k, v in metrics.items()}


#### Train model

In [46]:
# hyperparameters
lr = 2e-4
batch_size = 8
num_epochs = 5

import torch
use_mps = hasattr(torch.backends, "mps") and torch.backends.mps.is_available()

training_args = TrainingArguments(
    output_dir="bert-amazon-classifier",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,

    # ✅ Avoid W&B prompts unless you explicitly want them
    report_to="none",

    # ✅ Only enable MPS if available (Apple Silicon)
    use_mps_device=use_mps,
)


/opt/anaconda3/envs/work/lib/python3.12/site-packages/transformers/training_args.py:2266: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(


In [47]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],  # use validation during training
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


/var/folders/_9/749hzv410flf2x7ptjcpj0p80000gn/T/ipykernel_12271/514839692.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/opt/anaconda3/envs/work/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,1.267700,1.375806,0.800000,0.560000
2,1.259500,1.337318,0.800000,0.560000
3,1.228500,1.298277,0.800000,0.560000
4,1.180500,1.295261,0.800000,0.560000
5,1.152000,1.284703,0.800000,0.560000


/opt/anaconda3/envs/work/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/work/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/work/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/work/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=50, training_loss=1.21764066696167, metrics={'train_runtime': 8.2864, 'train_samples_per_second': 47.065, 'train_steps_per_second': 6.034, 'total_flos': 8359613086128.0, 'train_loss': 1.21764066696167, 'epoch': 5.0})

### Apply Model to Test Dataset

In [42]:
# Apply model to the held-out TEST dataset (unseen during training)
predictions = trainer.predict(tokenized_data["test"])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use compute_metrics
metrics = compute_metrics((logits, labels))
print(metrics)


/opt/anaconda3/envs/work/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'accuracy': 0.4, 'f1_macro': 0.25}


### Push to hub

In [15]:
# push model to hub
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/shawhin/bert-phishing-classifier_teacher/commit/0a93a9df5ee46d065e755326c7a75174e8f274c0', commit_message='End of training', commit_description='', oid='0a93a9df5ee46d065e755326c7a75174e8f274c0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/shawhin/bert-phishing-classifier_teacher', endpoint='https://huggingface.co', repo_type='model', repo_id='shawhin/bert-phishing-classifier_teacher'), pr_revision=None, pr_num=None)

### Run inference on new examples

In [45]:
# First, check if CUDA is available
import torch
use_mps = hasattr(torch.backends, "mps") and torch.backends.mps.is_available()
    
# Move your model to the appropriate device
model = model.to(device)

# Tokenize the input string
input_text = "Kitchen Storage Container Set for Rice"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

# Map prediction to label
predicted_label = model.config.id2label[predictions.item()]
print(f"Predicted label: {predicted_label}")

Predicted label: BOTTLE
